In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

SCRAPESTORM_API_KEY = os.getenv("SCRAPESTORM_API_KEY")
SCRAPESTORM_BASE_URL = os.getenv("SCRAPESTORM_BASE_URL")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/borislav/VSCode/CoinTracker/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
  File "/tmp/ipykernel_354156/140134159.py", line 2, in <module>
    from dotenv import load_dotenv
ModuleNotFoundError: No module named 'dotenv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/borislav/VSCode/CoinTracker/.venv/lib/python3.10/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/borislav/VSCode/CoinTracker/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2170, in showtraceback
  File "/home/borislav/VSCode/CoinTracker/.venv/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1457, in structured_traceb

In [119]:
import requests
import aiohttp
from typing import Literal, Dict, Any

TIMEOUT = 30

async def search_tweets_by_query(
    query: str, tag: Literal["top", "latest"] = "latest", cursor: str | None = None
) -> Dict[str, Any]:
    """Search tweets by query
    
    Args:
        query: Search query string
        tag: Type of tweets to fetch ('top' or 'latest')
        cursor: Pagination cursor for fetching next page of results
        
    Returns:
        Dict containing status, pagination info and tweet data
    """
    url = f"{SCRAPESTORM_BASE_URL}/twitter.com/api/v2.1/search_tweets_by_query/"
    params = {"token": SCRAPESTORM_API_KEY, "query": query, "tag": tag}
    if cursor:
        params["cursor"] = cursor
        
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url, params=params, timeout=TIMEOUT) as response:
                response.raise_for_status()
                return await response.json()
    except aiohttp.ClientError as e:
        print(f"Error searching tweets by query: {e}")
        raise

async def search_tweets(
    query: str, tag: Literal["top", "latest"] = "latest", num_pages: int = 1
) -> Dict[str, Any]:
    """Search tweets by query"""
    tweets = []
    cursor = None
    for _ in range(num_pages):
        result = await search_tweets_by_query(query, tag, cursor)
        tweets.extend(result["data"]['entries'])
        cursor = result["pagination"]["next_cursor"]
    return tweets

In [1]:
results

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/borislav/VSCode/CoinTracker/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
  File "/tmp/ipykernel_354156/962467392.py", line 1, in <module>
    results
NameError: name 'results' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/borislav/VSCode/CoinTracker/.venv/lib/python3.10/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/borislav/VSCode/CoinTracker/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2170, in showtraceback
  File "/home/borislav/VSCode/CoinTracker/.venv/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1457, in structured_traceback
  File "/home/borislav/V

In [ ]:
results = search_tweets('#memcoin', num_pages=3)

In [ ]:
from pydantic import BaseModel
from datetime import datetime, timezone


class TwitterUser(BaseModel):
    created_at: datetime
    is_verified: bool
    is_blue_verified: bool
    followers_count: int
    friends_count: int
    statuses_count: int
    has_photo: bool
    has_banner: bool
    
    @classmethod
    def from_scrapestorm_owner_record(cls, record: dict):
        return cls(
            created_at=datetime.fromisoformat(record['created_at']),
            is_verified=record['verified'],
            is_blue_verified=record['is_blue_verified'],
            followers_count=record['followers_count'],
            friends_count=record['friends_count'],
            statuses_count=record['statuses_count'],
            has_photo='profile_image_url' in record,
            has_banner='profile_banner_url' in record
        )

    @staticmethod
    def calculate_account_score(user: TwitterUser):
        score = 0.
        
        age_days = (datetime.now(timezone.utc) - user.created_at.replace(tzinfo=timezone.utc)).days
        
        if age_days < 7:
            score -= 5
        if age_days < 30:
            score -= 3
        elif age_days < 90:
            score -= 2
        elif age_days > 365:
            score += 1
        
        if user.is_verified:
            score += 10
        
        if user.is_blue_verified:
            score += 2
        
        if user.followers_count >= 500:
            score += 2
        elif user.followers_count >= 50:
            score += 1
        elif user.followers_count <= 10:
            score -= 1
        
        if user.friends_count == 0:
            score -= 1
            
        if user.followers_count == 0:
            score -= 1
        
        if user.friends_count > 0 and user.followers_count > 0:
            ratio = user.followers_count / user.friends_count
            if ratio < 0.2:
                score -= 1
            elif ratio > 5:
                score += 1
            
        if user.statuses_count >= 50:
            score += 1
        elif user.statuses_count <= 10:
            score -= 1
        
        if not user.has_photo:
            score -= 1
        
        if not user.has_banner:
            score -= 1
        
        return min(10, max(0, score))
        
    def is_spammer(self):
        return self.calculate_account_score(self) < 4        


In [95]:
id = 33
print(f"https://x.com/{results[id]['owner']['screen_name']}")
user = TwitterUser.from_scrapestorm_owner_record(results[id]['owner'])
user.is_spammer()

https://x.com/DCrypsi93016


True

In [ ]:
class Tweet(BaseModel):
    id: int
    text: str
    author: TwitterUser
    created_at: datetime
    views: int
    likes: int
    shares: int
    comments: int
    retweets: int
    has_attachment: bool

    @classmethod
    def from_scrapestorm_tweet(cls, tweet: dict):
        return cls(
            id=tweet['id'],
            text=tweet['description'],
            author=TwitterUser.from_scrapestorm_owner_record(tweet['owner']),
            created_at=datetime.fromisoformat(tweet['created_at']),
            views=tweet['view_count'],
            likes=tweet['liked_count'],
            shares=tweet['share_count'],
            comments=tweet['comment_count'],
            retweets=tweet['retweet_count'],
            has_attachment=len(tweet['contents']) > 0
        )
        
    def is_spam(self):
        return self.author.is_spammer()

In [102]:
Tweet.from_scrapestorm_tweet(results[0])

Tweet(id=1938954160571138194, text='@nefitati @ApeCryptos @XOriginals #XOriginals the best memcoin ever https://t.co/LeMQJTCJpv', author=TwitterUser(created_at=datetime.datetime(2024, 11, 12, 9, 36, 54), is_verified=False, is_blue_verified=False, followers_count=47, friends_count=0, statuses_count=1003, has_photo=True, has_banner=True), created_at=datetime.datetime(2025, 6, 28, 13, 34, 30), views=4, likes=0, shares=0, comments=0, retweets=0, has_attachment=True)

In [107]:
for result in results:
    tweet = Tweet.from_scrapestorm_tweet(result)
    if not tweet.is_spam():
        print('###', tweet.text)

### Cool game on #bsc

CZ mentioned this memcoin - see for yourself

🚨 Alert &amp; 🌪️Quick buy ⤵️
https://t.co/Cm8KtihlDI

$Binanciens

0x02367c2b93fbcbcc6a11f39fcb50f713c4cb4444

#Binanciens https://t.co/IapLrZgB2K
### @TimHaldorsson U mean sh!t memcoin rn ?
### Basic: Meme coin website, presale landing page on all chain!
Standard: presale smart contract &amp; integrate web3 feature!
Premium: ICO launchpad pump fun, pinksale fork
Choose your Package!
Place order! https://t.co/n9uL1ppYW7

#crypto #memcoin #web3 #EVM #Solana #tokenpresale #TON https://t.co/4wgEMEFzha
### @Shad0w_0G @MemeX_intern @MemeX_MRC20 Memex building the next memcoin infa
### Memecoin cơ hội hay trò chơi? #memcoin #meme #coin #crypto #blockchain #tiendientu https://t.co/015krMbuCK
### What's the next 100x? 

I would say $yapyo but on Arbitrum!

Wouldn't you agree? 

Yapyo maybe launching on Gu Factory @gu_trade as it supports both Yapyo and Arbitrum accounts. Gu Factory is Arbitrum based memcoin token launcher, an

In [124]:
import asyncio

keywords = [
    'memecoin', 'shitcoin', 'PumpFun', 'airdrop', 'meme AND coin', 'shit AND coin'
]

# search tweets for keywords

results = []

for keyword in keywords:
    print(f"Searching for {keyword}")
    found = await search_tweets(keyword)
    print(f"Found {len(found)} tweets")
    results.extend(found)

Searching for memecoin
Found 12 tweets
Searching for shitcoin
Found 18 tweets
Searching for PumpFun
Found 12 tweets
Searching for airdrop
Found 11 tweets
Searching for meme AND coin
Found 12 tweets
Searching for shit AND coin
Found 13 tweets


In [152]:
# есть ли упоминание конкретной монеты
# является ли она мем коином или альткоином
# имя монеты
# какой сентимент у поста

from pydantic import BaseModel, Field
from typing import Literal


class CoinTweetAnalysis(BaseModel):
    is_speculative_coin: bool = Field(description="Is the coin a speculative coin?")
    coin_name: str = Field(description="The name of the coin")
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="The sentiment of the tweet")
    keywords: list[str] = Field(description="2-3 keywords from the tweet")
    

In [155]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0,
)
model_with_structured_output = model.with_structured_output(CoinTweetAnalysis)


PROMPT = """
You are a crypto-savvy analyst.  
Given **one tweet** about crypto, extract structured information and return it **exactly** in the JSON schema shown below.

# ➊ Identify whether a **specific cryptocurrency** is mentioned  
• Look only for explicit tickers (`$PEPE`, `PEPE/USDT`, `#PEPE`) **or** well-known coin names (Bitcoin, Solana, XRP, etc.).  
• Ignore generic words like “shitcoin” when no concrete ticker follows.  
• If no specific coin is found, leave `coin_name` an empty string and set `is_speculative_coin` to **false**.

# ➋ Determine the most important coin (`coin_name`)  
• If several $-tickers appear, choose the one that appears **earliest** in the tweet.  
• Return only the **uppercase** ticker (strip `$`, `#`, or pair suffixes like `/USDT`).

# ➌ Classify `is_speculative_coin` (boolean)  
Set **true** if the selected coin falls in any of these categories:  
1. **Memecoin** by reputation (DOGE, PEPE, SHIB, FLOKI, BONK, BART, MOOP, etc.).  
2. **Shitcoin** (the tweet explicitly calls the coin a “shitcoin” or similar slur).  
3. **Altcoin**—i.e., any smaller-cap or mid-cap crypto that is **not** among the top blue chips (BTC or ETH).  
A hashtag like `#memecoin` alone is **not** enough; decide based on the coin itself.  
If no coin or the coin is BTC/ETH (or another blue-chip) → `is_speculative_coin: false`.

# ➍ Sentiment (`positive` | `negative` | `neutral`)  
• Use overall tone.  
• Sarcasm: if the surface text is praise but includes cues like “yeah right”, “LMAO”, treat as **negative**.

# ➎ Keywords  
• Provide **2–3** concise nouns or noun-phrases that describe the coin’s context (e.g., “pump”, “creator rewards”, “airdrop”).  
• **Do not include** generic trading boiler-plate: vip, tp, signal, invest, join, trading, forex, pumpfun, pump, TP5/TP10, giveaways, “join telegram”, etc.

# ➏ Output  
Return **only** the following JSON object (no extra keys, no commentary; booleans in lowercase):

```json
{
  "is_speculative_coin": true | false,
  "coin_name": "TICKER" | "",
  "sentiment": "positive" | "negative" | "neutral",
  "keywords": ["kw1", "kw2", "kw3"]
}
````

# 📌 Example

**Input tweet**

> I'm seeing a lot of hype around \$DOGE lately, is it still a good buy?

**Expected JSON**

```json
{
  "is_speculative_coin": true,
  "coin_name": "DOGE",
  "sentiment": "neutral",
  "keywords": ["hype", "buy", "DOGE"]
}
```

---

### Now analyze this tweet:

```
{tweet}
```
"""


# model_with_structured_output.invoke(PROMPT.format(tweet=tweet))


In [156]:
analysis_results = []

for result in results:
    tweet = Tweet.from_scrapestorm_tweet(result)
    if not tweet.is_spam():
        text = tweet.text.replace("\n", " ")
        analysis = model_with_structured_output.invoke(PROMPT.replace("{tweet}", text))
        print('###')
        print(text)
        print(analysis)
        analysis_results.append(analysis)
        

###
#SPX6900 will redefine what Memecoin is. https://t.co/QknaMOzoaG
is_speculative_coin=True coin_name='SPX6900' sentiment='positive' keywords=['Memecoin', 'redefine', 'SPX6900']
###
$SEI ✅  #Altseason  #Cryptocurency #XRP $FOLKI #Solana #bullish #AI #memecoin Join our telegram community. Click below ⬇️ https://t.co/4KyDq2xuTP https://t.co/HFe2xoVgnh
is_speculative_coin=True coin_name='SEI' sentiment='positive' keywords=['Altseason', 'bullish', 'community']
###
#BSWUSDT TP10✅  PM ME TO JOIN VIP AND INVEST: https://t.co/JwEieonpjs  #bitcoin  #crypto #btc #eth #altcoin #trading #signal #memecoin #forex #xau #investing #gold #blockchain $BSW https://t.co/dWNGgxzGSs
is_speculative_coin=True coin_name='BSW' sentiment='neutral' keywords=['BSW', 'blockchain', 'altcoin']
###
#LISTAUSDT TP9✅  PM ME TO JOIN VIP AND INVEST: https://t.co/JwEieonpjs  #bitcoin  #crypto #btc #eth #altcoin #trading #signal #memecoin #forex #xau #investing #gold #blockchain $LISTA https://t.co/CLvhBxcbmY
is_speculativ